In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

import timm
from torchvision import transforms
from sklearn.model_selection import KFold
from torch.cuda.amp import autocast, GradScaler


In [3]:
class Config:
    train_path = "/kaggle/input/csiro-biomass/train.csv"
    test_path = "/kaggle/input/csiro-biomass/test"
    sample_sub = "/kaggle/input/csiro-biomass/sample_submission.csv"
    root_dir = "/kaggle/input/csiro-biomass"
    sample_id = "ID1001187975__Dry_Clover_g"
    TARGET_NAMES = [
        "Dry_Green_g",
        "Dry_Dead_g",
        "Dry_Clover_g",
        "GDM_g",
        "Dry_Total_g",
    ]


config = Config()


def weighted_r2_score(y_true, y_pred):
        weights = np.array([0.1, 0.1, 0.1, 0.2, 0.5])
        r2s = []
    
        for i in range(5):
            yt, yp = y_true[:, i], y_pred[:, i]
            ss_res = np.sum((yt - yp) ** 2)
            ss_tot = np.sum((yt - yt.mean()) ** 2)
            r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
            r2s.append(r2)
    
        r2s = np.array(r2s)
        return np.sum(r2s * weights) / weights.sum(), r2s




In [4]:
train_df = pd.read_csv(config.train_path)

train_df.head(5)

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


In [5]:
df = train_df.pivot(
    index="image_path",
    columns="target_name", 
    values="target"
).reset_index()

df.columns.name = None 

df.head(5)

,image_path,Dry_Clover_g,Dry_Dead_g,Dry_Green_g,Dry_Total_g,GDM_g
0,train/ID1011485656.jpg,0.0000,31.9984,16.2751,48.2735,16.2750
1,train/ID1012260530.jpg,0.0000,0.0000,7.6000,7.6000,7.6000
2,train/ID1025234388.jpg,6.0500,0.0000,0.0000,6.0500,6.0500
3,train/ID1028611175.jpg,0.0000,30.9703,24.2376,55.2079,24.2376
4,train/ID1035947949.jpg,0.4343,23.2239,10.5261,34.1844,10.9605


In [6]:
class BiomassDataset(Dataset):
    def __init__(self, df, root_dir, transform):
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.root_dir, row["image_path"])).convert("RGB")
        img = self.transform(img)

        target = torch.tensor([
            row["Dry_Green_g"],
            row["Dry_Dead_g"],
            row["Dry_Clover_g"],
            row["GDM_g"],
            row["Dry_Total_g"]
        ], dtype=torch.float32)

        return img, target



In [7]:
train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [8]:
class ConvNeXtRegressor(nn.Module):
    def __init__(self, num_targets=5):
        super().__init__()

        self.backbone = timm.create_model(
            "convnext_tiny",
            pretrained=True,
            features_only=False
        )
        self.backbone.head = nn.Identity()

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Linear(768, num_targets)

    def forward(self, x):
        x = self.backbone.forward_features(x)
        x = self.pool(x).flatten(1)
        return self.head(x)


In [9]:
def train_one_epoch(model, loader, optimizer, criterion, scaler, device):
    model.train()
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        with autocast():
            preds = model(x)
            loss = criterion(preds, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * x.size(0)

    return total_loss / len(loader.dataset)


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds, trues = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            p = model(x)

            total_loss += criterion(p, y).item() * x.size(0)
            preds.append(p.cpu())
            trues.append(y.cpu())

    preds = torch.cat(preds).numpy()
    trues = torch.cat(trues).numpy()

    wr2, _ = weighted_r2_score(trues, preds)
    return total_loss / len(loader.dataset), wr2


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"


kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (tr_idx, va_idx) in enumerate(kf.split(df)):
    print(f"\n===== Fold {fold+1} =====")

    train_ds = BiomassDataset(df.iloc[tr_idx], config.root_dir, train_tfms)
    val_ds   = BiomassDataset(df.iloc[va_idx], config.root_dir, val_tfms)

    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
    val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=2)

    model = ConvNeXtRegressor().to(device)
    criterion = nn.SmoothL1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    scaler = GradScaler()

    best_r2 = -1

    for epoch in range(10):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler, device)
        val_loss, val_r2 = validate(model, val_loader, criterion, device)
        scheduler.step()

        print(f"Epoch {epoch+1}: Train {train_loss:.4f} | Val {val_loss:.4f} | R2 {val_r2:.4f}")

        if val_r2 > best_r2:
            best_r2 = val_r2
            torch.save(model.state_dict(), f"best_convnext_fold{fold}.pth")



===== Fold 0 =====


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Epoch 1: Train 17.2206 | Val 12.9463 | R2 -0.1123
Epoch 2: Train 12.9558 | Val 12.0118 | R2 0.1102
Epoch 3: Train 11.9523 | Val 11.0985 | R2 0.2639
Epoch 4: Train 11.1660 | Val 9.1224 | R2 0.3509
Epoch 5: Train 10.0590 | Val 9.1019 | R2 0.4341
Epoch 6: Train 9.8837 | Val 8.0522 | R2 0.5106
Epoch 7: Train 8.4177 | Val 9.1471 | R2 0.4863
Epoch 8: Train 7.4780 | Val 7.8158 | R2 0.5249
Epoch 9: Train 6.8556 | Val 7.6056 | R2 0.5828
Epoch 10: Train 6.4479 | Val 7.1627 | R2 0.5995

===== Fold 1 =====
Epoch 1: Train 15.9379 | Val 14.2444 | R2 0.0357
Epoch 2: Train 12.0668 | Val 14.1694 | R2 0.1175
Epoch 3: Train 11.3486 | Val 13.6263 | R2 -0.0294
Epoch 4: Train 10.2387 | Val 12.0870 | R2 0.1918
Epoch 5: Train 9.3385 | Val 12.1167 | R2 0.3170
Epoch 6: Train 8.4874 | Val 10.5415 | R2 0.3097
Epoch 7: Train 7.9967 | Val 9.8787 | R2 0.3877
Epoch 8: Train 6.9945 | Val 9.6385 | R2 0.4854
Epoch 9: Train 6.5361 | Val 9.2224 | R2 0.4646
Epoch 10: Train 6.1417 | Val 8.8870 | R2 0.5256

===== Fold 2 ====